<a href="https://colab.research.google.com/github/haroldgo23/ph-fake-news-detection/blob/main/Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
import pandas as pd
import numpy as np
import re
import string

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df_eng = pd.read_csv("/content/drive/MyDrive/Machine Learning Tool for Identification of Philippine Fake News in Social Media/English.csv")
df_fil = pd.read_csv("/content/drive/MyDrive/Machine Learning Tool for Identification of Philippine Fake News in Social Media/Filipino.csv")

Mounted at /content/drive


# Preparing the Dataset

In [3]:
df_merge = pd.concat([df_eng, df_fil], axis =0 )
df = df_merge.drop(["Page"], axis = 1) # combine 
df = df.sample(frac = 1) # shuffle dataset
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)
df.head()

,Text,Label
0,According to an article by Betheena Unite of M...,1
1,"Saab Magalona, daughter of the King of Pinoy R...",1
2,Pasensya ang nakikitang solusyon nina Pangulon...,0
3,Ang meaning ng karaniwang idioms or phrases ay...,1
4,"Noel Landero Sarifa, if you are a true Duterte...",1


### Define dependent and independent variables

In [4]:
x = df["Text"]
y = df["Label"]

### Remove extra space etc, lower case

In [5]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [6]:
df["Text"] = df["Text"].apply(wordopt)  #apply wordopt function

# Train-test split

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Convert text to vectors

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
X = vectorization.fit_transform(x)
xv_train = vectorization.transform(x_train)
xv_test = vectorization.transform(x_test)

# Import Models

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Model Hyperparameters

### LR

In [10]:
param_lr = [{
                'penalty': [None, 'l2'], 
                'C': [0.1, 1, 10, 100, 1000],
                'solver': ['lbfgs', 'newton-cg', 'sag'], 
                'max_iter': [100, 500, 1000, 10000]
            },
            {
                'penalty': ['l1', 'l2'], 
                'C': [0.1, 1, 10, 100, 1000],
                'solver': ['liblinear'], 
                'max_iter': [100, 500, 1000, 10000]
            },
            {
                'penalty': [None, 'l1', 'l2', 'elasticnet'], 
                'C': [0.1, 1, 10, 100, 1000],
                'solver': ['saga'], 
                'max_iter': [100, 500, 1000, 10000]
            }]

### DTC

In [11]:
param_dtc = {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_depth': [None, 2, 3, 4, 5, 6],
            'min_samples_split': [2, 4, 6],
            'max_features': [None, 'auto', 'sqrt', 'log2'],
            'min_samples_leaf': [1, 2]
            }

### RFC

In [12]:
param_rfc= {
            'n_estimators': [10, 25, 50, 100, 200],
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_depth': [None, 2, 3, 4, 5, 6],
            'min_samples_split': [2, 4, 6],
            'max_features': [None, 'sqrt', 'log2'],
            'min_samples_leaf': [1, 2]
            }

### GBC

In [13]:
param_gbc = {
            'learning_rate': [0.01, 0.02, 0.03, 0.04],
            'n_estimators': [100, 500, 1000],
            'subsample': [1.0, 0.8, 0.5, 0.2],
            'max_depth': [3, 4, 6, 8]
             }

### PAC

In [14]:
param_pac = {
            'C': [0.1, 1, 10, 100, 1000],
            'max_iter': [100, 500, 1000]
             }

### NB

In [15]:
param_nb =  {
            'alpha': [0]
            #'force_alpha': [True]
             }

### SVM

In [16]:
param_svm = {
            'C':[0.1, 1, 10, 100, 1000],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'degree': [0.001, 0.01, 0.1, 1]
             }

# RandomizedSearchCV

### LR

In [17]:
lr = RandomizedSearchCV(LogisticRegression(), param_lr , cv=10, return_train_score=False, n_iter=100)
lr = lr.fit(xv_train, y_train)
lr.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

{'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 500, 'C': 1000}

In [18]:
acc_lr = lr.best_score_
acc_lr

0.9712810559006211

### DTC

In [19]:
dtc = RandomizedSearchCV(DecisionTreeClassifier(), param_dtc , cv=10, return_train_score=False, n_iter=100)
dtc = dtc.fit(xv_train, y_train)
dtc.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
310 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
310 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_fits_failed_message, FitFa

{'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': None,
 'max_depth': None,
 'criterion': 'entropy'}

In [20]:
acc_dtc = dtc.best_score_
acc_dtc

0.898889751552795

### PAC

In [21]:
pac = RandomizedSearchCV(PassiveAggressiveClassifier(), param_pac , cv=10, return_train_score=False, n_iter=10)
pac = pac.fit(xv_train, y_train)
pac.best_params_

{'max_iter': 1000, 'C': 10}

In [22]:
acc_pac = pac.best_score_
acc_pac

0.9700271739130436

### SVM

In [23]:
svm = RandomizedSearchCV(svm.SVC(probability=True), param_svm , cv=10, return_train_score=False, n_iter=10)
svm = svm.fit(xv_train, y_train)
svm.best_params_

{'kernel': 'linear', 'degree': 0.001, 'C': 100}

In [24]:
acc_svm = svm.best_score_
acc_svm

0.9744060559006211

### NB

In [25]:
nb = RandomizedSearchCV(MultinomialNB(), param_nb , cv=10, return_train_score=False)
nb = nb.fit(xv_train, y_train)
nb.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small

{'alpha': 0}

In [26]:
acc_nb = nb.best_score_
acc_nb

0.9500621118012423

### RFC

In [27]:
rfc = RandomizedSearchCV(RandomForestClassifier(), param_rfc , cv=10, return_train_score=False, n_iter=100)
rfc = rfc.fit(xv_train, y_train)
rfc.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
390 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
390 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py

{'n_estimators': 50,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': None,
 'criterion': 'gini'}

In [28]:
acc_rfc = rfc.best_score_
acc_rfc

0.9625465838509317

### GBC

In [29]:
gbc = RandomizedSearchCV(GradientBoostingClassifier(), param_gbc , cv=10, return_train_score=False, n_iter=10)
gbc = gbc.fit(xv_train, y_train)
gbc.best_params_

{'subsample': 1.0, 'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.02}

In [30]:
acc_gbc = gbc.best_score_
acc_gbc

0.9744099378881987